# Sequence Alignment
Optimization problem, pairwise sequence alignment, two sequences (finite alphabet), objective function.

The end goal is an optimal alignment (maximizes the objective function)

The objective function is additive for (DNA & RNA):
 * `matches: +2/+3` (positive contribution)
 * `mismatches: -2/-3` (replacement, negative contribution)
 * `gaps: -2/-3`(higher impact than replacement, negative contribution)
 
 Substitution matrix -> assigns a value for each pair, does not consider gaps